In [2]:


import pandas as pd
from gensim.summarization import keywords
import matplotlib.pyplot as plt
df =pd.read_csv('data.csv')
jd = df['Job Description'].tolist()
companies = df['company'].tolist()
positions = df['position'].tolist()

from gensim import models
docs = []
for i in range(len(jd)):
    sent = models.doc2vec.LabeledSentence(words = jd[i].split(),tags = ['{}_{}'.format(companies[i], i)])
    #sent = models.doc2vec.LabeledSentence(words = jd[i].split(),tags = ['jd'])
    docs.append(sent)
    
model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)
model.build_vocab(docs)

for epoch in range(10):
    model.train(docs)
    model.alpha -= 0.002  # decrease the learning rate`
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    
with open('resumeconverted.txt','r') as f:
    resume = f.read()

    
from sklearn.manifold import MDS
data = []
for i in range(len(jd)):
    data.append(model.docvecs[i])

data.append(model.infer_vector(resume))

mds = MDS(n_components=2, random_state=1)
pos = mds.fit_transform(data)
#print pos
xs,ys = pos[:,0], pos[:,1]
for x, y in zip(xs, ys):
    plt.scatter(x, y)
#    plt.text(x, y, name)
xs2,ys2 = xs[-1], ys[-1]
plt.scatter(xs2, ys2, c='Red', marker='+')
plt.text(xs2,ys2,'resume')
plt.savefig('distance.png')
plt.show()


from sklearn.metrics.pairwise import cosine_distances
cos_dist =[]
for i in range(len(data)-1):
    cos_dist.append(float(cosine_distances(model.infer_vector(resume),data[i])))
    

key_list =[]

for j in jd:
    key =''
    for word in keywords(j).split('\n'):
        key += '{} '.format(word)
    key_list.append(key)

summary = pd.DataFrame({
        'Company': companies,
        'Postition': positions,
        'Cosine Distances': cos_dist,
        'Keywords': key_list,
        'Job Description': jd
    })
z =summary.sort('Cosine Distances', ascending=False)
z.to_csv('Summary.csv',encoding="utf-8")

ModuleNotFoundError: No module named 'gensim'

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 21 22:13:01 2017

@author: binoy
"""
from bs4 import BeautifulSoup
import json
import urllib
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

with open('url.json','r') as f:
    url = json.load(f)
data ={}    
    
i = 1
jd_df = pd.DataFrame()

driver = webdriver.Firefox(executable_path=r'C:\Tools\geckodriver-v0.16.0-win64\geckodriver.exe')


for u in url:
    driver.wait = WebDriverWait(driver, 5)
    driver.maximize_window()
    driver.get(u)
#    r = urllib.urlopen(u).read()
    soup = BeautifulSoup(driver.page_source, "lxml")
#    jd_df['position'] =
#    print soup
    header = soup.find("div",{"class":"header cell info"})
    position =header.find('h2').get_text()
    company = header.find("span",{"class":"ib"}).get_text()
    location = header.find("span",{"class": "subtle ib"}).get_text()[2:]
    jd = soup.find("div",{"class":"jobDescriptionContent desc"}).get_text()
#    website = soup.find("span",{"class":"value website"}).get_text()
    info = soup.find_all("div",{"class":"infoEntity"})
    try:    
        headquaters = info[1].find("span",{"class":"value"}).get_text().strip().strip('\u')
        employees = info[2].find("span",{"class":"value"}).get_text().strip().strip('\u')
        founded = info[3].find("span",{"class":"value"}).get_text().strip().strip('\u')
        industry = info[5].find("span",{"class":"value"}).get_text().strip().strip('\u')
    except:
        heardquaters = None
        employees = None
        founded = None
        industry = None
#    revenue = info[6].find("span",{"class":"value"}).get_text().strip()
#    competitors = info[7].find("span",{"class":"value"}).get_text().strip()    
    data[i] = {
        'url' :u,
        'company': company,
        'position': position,
        'location' :location,
#        'website':website,
        'headquaters':headquaters,
        'employees':employees,
        'founded' :founded,
        'industry' :industry,
#        'revenue' :revenue,
#        'competitors' :competitors,
        'Job Description' :jd
    }
    print i
    i+=1 
    
driver.quit()
jd_df = pd.DataFrame(data)
jd = jd_df.transpose()
#cols = jd.columns.tolist()
#print cols
jd = jd[['company','position','url','location','headquaters','employees','founded','industry','Job Description']]
jd.to_csv('data.csv',encoding="utf-8")

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (<ipython-input-3-bbec90fb2cef>, line 69)